# NIRISS AMI: Pipeline

<font color="red">This notebook currently fails to execute, use as reference only</font>

**Use case:** Run pipeline and stand-alone tool ImPlaneIA on NIRISS AMI data.<br>
**Data:** JWST data from commissioning.<br>
**Tools:**  jwst, astropy.<br>
**Cross-intrument:** <br>
**Documentation:** This notebook is part of a STScI's larger [post-pipeline Data Analysis Tools Ecosystem](https://jwst-docs.stsci.edu/jwst-post-pipeline-data-analysis).<br>

**Latest update**: December 2022

## Introduction
This notebook runs JWST pipeline on Aperture Masking Interferometry(AMI) data of binary point source AB Dor and calibrator HD37093 observed during AMI commissioning. We are only using two NRM + F480M exposures at dither position POS1, one for the target and one for the calibrator.

Steps:

[1] Run Detector1 pipeline on all _uncal.fits files to create _rate.fits and _rateints.fits files.

[2] Run Image2 pipeline on all _rate.fits files to create _cal.fits and on _rateints.fits files to
   create _calints.fits files.

 
[3] Run ImPlaneIA ([Greenbaum, A. et al. 2015](https://ui.adsabs.harvard.edu/abs/2015ApJ...798...68G/abstract)) to extract observables in oifits format.    

***

## Imports
Describe the libraries we're using here. If there's something unusual, explain what the library is, and why we need it.
- *numpy* to handle array functions
- *astropy.io fits* for accessing FITS files
- *matplotlib.pyplot* for plotting data
- *zipfile* for accessing zip file
- *urllib.request* to access URL
- *jwst.pipeline Detector1Pipeline, Image2Pipeline* for calibrating raw data
- ImplaneIA to extract interferometric obssrvables from calibrated data

This notebook requires a series of reference data to run. These must be set at the very beginning of the notebook, before importing the relative packages. In some cases, these locations are already provided in shall configuration files. If this is not the case, they can be set here. Follow the instruction to download the required data:
- PYSYN_CDBS: https://pysynphot.readthedocs.io/en/latest/#installation-and-setup
- WEBBPSF_PATH: https://webbpsf.readthedocs.io/en/latest/installation.html#installing-the-required-data-files
- CRDS_PATH and CRDS_SERVER_URL: https://hst-crds.stsci.edu/static/users_guide/environment.html

In [ ]:
# Modify the path to a directory on your machine
import os
# os.environ["CRDS_PATH"] = ""
os.environ["CRDS_SERVER_URL"] = "https://jwst-crds.stsci.edu"

# WEBBPSF and STSYNPHOT
# os.environ['WEBBPSF_PATH'] = ""
# os.environ['PYSYN_CDBS'] = ""

In [ ]:
# flake8-ignore: E402
# imports
%matplotlib inline
import glob
import os
from pathlib import Path

import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
import zipfile
import urllib.request

from jwst.pipeline import Detector1Pipeline, Image2Pipeline

from nrm_analysis.misctools import utils
from nrm_analysis import nrm_core, InstrumentData
from nrm_analysis.misctools.implane2oifits import calibrate_oifits

from run_bp_fix import correct_fitsfiles

## Loading data
Download the data of the AMI commissioning activity:  
Ab Dor: jw01093012001_03102_00001_nis_uncal.fits  
HD37093: jw01093015001_03102_00001_nis_uncal.fits

In [ ]:
# Download the data of the AMI commissioning activity
boxlink = 'https://data.science.stsci.edu/redirect/JWST/jwst-data_analysis_tools/niriss_ami_binary/niriss_ami_binary2_inflight.zip'
boxfile = Path('./niriss_ami_binary2_inflight.zip')

# Download zip file
if not os.path.exists(boxfile):
    urllib.request.urlretrieve(boxlink, boxfile)

    zf = zipfile.ZipFile(boxfile, 'r')
    zf.extractall()

In [ ]:
# Define directory that has commissioning data. NRM + F480M exposures of AB Dor, HD 37093 at POS 1
currentdir = Path('.')
inflightdata = currentdir / 'niriss_ami_binary2_inflight'
datafiles = list(sorted(inflightdata.glob('jw*uncal.fits')))
for i in datafiles:
    print(i)

## Examine the input raw files

Look at the last group of the first integration of the uncal.fits file

In [ ]:
data = []
for i, df in enumerate(datafiles):
    file = fits.open(df)
    file.info()
    im = file[1].data
    print(im[0].shape)
    header = file[0].header
    print(header['TARGPROP'])
    data.append(im[0])
print(data[0].shape, data[1].shape)
f = plt.figure(figsize=(12, 6))
plt.suptitle("NRM + F480M raw exposures (last group of integration 1) at POS1", fontsize=18, fontweight='bold')
# display the last group of the first integration of each file
plt.subplot(1, 2, 1)
plt.title("AB Dor")
plt.imshow(data[0][4], origin='lower')
plt.subplot(1, 2, 2)
plt.title("HD37093")
plt.imshow(data[1][11], origin='lower')

## Define output directory

In [ ]:
# Define output directory to save pipeline output products.
outdir = Path('./pipeline_calibrated_data/')
if not os.path.exists(outdir):
    os.mkdir(outdir)
    print("Created", outdir)

## Run Detector1 and Image2 pipelines

In [ ]:
# Run Detector1, Image 2 pipelines
for df in datafiles:
    result1 = Detector1Pipeline()
    # Example code to override reference files
    # superbiasfile = refdir + 'jwst_niriss_superbias_sim.fits'
    # darkfile = refdir + 'jwst_niriss_dark_sub80_sim.fits'
    # result1.superbias.override_superbias = superbiasfile
    # result1.dark_current.override_dark = darkfile
    result1.ipc.skip = True
    result1.save_results = True
    result1.save_calibrated_ramp = True
    result1.output_dir = str(outdir)
    result1.run(str(df))

    df_rate = outdir / df.name.replace('uncal', 'rate')
    result2 = Image2Pipeline()
    # Example code to override reference files
    # flatfieldfile = refdir + "jwst_niriss_flat_general.fits"
    # result2.flat_field.override_flat = flatfieldfile
    result2.photom.skip = True
    result2.resample.skip = True
    result2.save_results = True
    result2.output_dir = str(outdir)
    result2.run(str(df_rate))

    df_rateints = outdir / df.name.replace('uncal', 'rateints')
    result3 = Image2Pipeline()
    # Example code to override reference files
    # result3.flat_field.override_flat = flatfieldfile
    result3.photom.skip = True
    result3.resample.skip = True
    result3.save_results = True
    result3.output_dir = str(outdir)
    result3.run(str(df_rateints))

## Examine the output files

In [ ]:
rampfiles = sorted(glob.glob(str(outdir / 'jw*ramp.fits')))
print("\n".join(rampfiles))
ratefiles = sorted(glob.glob(str(outdir / 'jw*rate.fits')))
print("\n".join(ratefiles))
rateintsfiles = sorted(glob.glob(str(outdir / 'jw*rateints.fits')))
print("\n".join(rateintsfiles))
calfiles = sorted(glob.glob(str(outdir / 'jw*cal.fits')))
print("\n".join(calfiles))
calintsfiles = sorted(glob.glob(str(outdir / 'jw*calints.fits')))
print("\n".join(calintsfiles))

### rate and rateints files

In [ ]:
for i, rateintf in enumerate(rateintsfiles):
    fits.info(rateintf)
for i, ratef in enumerate(ratefiles):
    fits.info(ratef)

### cal and calints files 

In [ ]:
for i, calintf in enumerate(calintsfiles):
    fits.info(calintf)
for i, calf in enumerate(calfiles):
    fits.info(calf)

## Display calibrated data

In [ ]:
data = []
for df in calintsfiles:
    print(df)
    im = fits.getdata(df, ext=1)    
    print(im.shape)
    data.append(im)
# print(data[0].shape, data[1].shape)
f = plt.figure(figsize=(12, 6))
plt.suptitle("NRM + F480M calibrated exposures (first integration) at POS1", fontweight='bold', fontsize=20)
# Look at the first integration from each calints.fits file
plt.subplot(1, 2, 1)
plt.title("AB Dor")
plt.imshow(data[0][0], clim=(-6000, 40000), origin='lower')
plt.subplot(1, 2, 2)
plt.title("HD37093")
plt.imshow(data[1][0], clim=(-6000, 40000), origin='lower')

### Fix bad pixels

In [ ]:
datasuperdir = Path('./pipeline_calibrated_data_corr/')

correct_fitsfiles(indir=outdir,
                  odir=datasuperdir)

calintfiles_corr = sorted(glob.glob(str(datasuperdir / '*calints.fits')))
print("\n".join(calintfiles_corr))

##  Display calibrated data after fixing bad pixels

In [ ]:
data = []
for df in calintfiles_corr:
    file = fits.open(df)
    im = file[1].data
    print(im.shape)
    data.append(im[0])
# print(data[0].shape, data[1].shape)
f = plt.figure(figsize=(12, 6))
# plt.tight_layout()
plt.subplot(1, 2, 1)
plt.suptitle("NRM + F480M calibrated exposures at POS1 (after fixing bad pixels)", fontweight='bold', fontsize=20)
plt.title("AB Dor")
plt.imshow(data[0], clim=(-6000, 40000), origin='lower')
plt.subplot(1, 2, 2)
plt.title("HD 37093")
plt.imshow(data[1], clim=(-6000, 40000), origin='lower')

## Run ImPlaneIA to reduce calibrated images to fringe observables

### Define functions

In [ ]:
np.set_printoptions(precision=4, linewidth=160)


def examine_observables(ff, trim=36):
    """ input: FringeFitter instance after fringes are fit """
   
    print("\nExamine_observables, standard deviations & variances of *independent* CP's and CAs:")
    print("   Closure phase mean {:+.4f}  std dev {:.2e}  var {:.2e}".format(ff.nrm.redundant_cps.mean(),
          np.sqrt(utils.cp_var(ff.nrm.N, ff.nrm.redundant_cps)), utils.cp_var(ff.nrm.N, ff.nrm.redundant_cps)))

    print("   Closure amp   mean {:+.4f}  std dev {:.2e}  var {:.2e}".format(ff.nrm.redundant_cas.mean(),
          np.sqrt(utils.cp_var(ff.nrm.N, ff.nrm.redundant_cas)), utils.cp_var(ff.nrm.N, ff.nrm.redundant_cas)))

    print("   Fringe amp   mean {:+.4f}  std dev {:.2e}  var {:.2e}".format(ff.nrm.fringeamp.mean(),
                                                                            ff.nrm.fringeamp.std(), 
                                                                            ff.nrm.fringeamp.var()))

    np.set_printoptions(precision=3, formatter={'float': lambda x: '{:+.1e}'.format(x)}, linewidth=80)
    print(" Normalized residuals central 6 pixels")
    tlo, thi = (ff.nrm.residual.shape[0]//2 - 3, ff.nrm.residual.shape[0]//2 + 3)
    print((ff.nrm.residual/ff.datapeak)[tlo:thi, tlo:thi])
    print(" Normalized residuals max and min: {:.2e}, {:.2e}".format(ff.nrm.residual.max() / ff.datapeak,
                                                                     ff.nrm.residual.min() / ff.datapeak))
    utils.default_printoptions()


def raw_observables(fitsfn=None, fitsimdir=None, oitdir=None, oifdir=None, affine2d=None,                    
                    psf_offset_find_rotation=(0.0, 0.0),
                    psf_offset_ff=None,
                    rotsearch_d=None,
                    set_pistons=None,
                    oversample=3,
                    mnem='',
                    firstfew=None,
                    usebp=False,
                    verbose=False):
    """
        Reduce calibrated data to fringe observables

        returns: affine2d (measured or input),
        psf_offset_find_rotation (input),
        psf_offset_ff (input or found),
        fringe pistons/r (found)
    """

    if verbose:
        print("raw_observables: input", fitsimdir / fitsfn)
    if verbose:
        print("raw_observables: oversample", oversample)

    fobj = fits.open(fitsimdir / fitsfn)

    if verbose:
        print(fobj[0].header['FILTER'])
        
    niriss = InstrumentData.NIRISS(fobj[0].header['FILTER'],
                                   usebp=usebp,
                                   firstfew=firstfew, # read_data truncation to only read first few slices...
                                   )

    ff = nrm_core.FringeFitter(niriss, 
                               oitdir=str(oitdir), # write OI text files here, and diagnostic images if desired
                               oifdir=str(oifdir), # write OI fits files here
                               oversample=oversample,
                               interactive=False,
                               save_txt_only=False)

    ff.fit_fringes(str(fitsimdir / fitsfn))
    examine_observables(ff)

    np.set_printoptions(formatter={'float': lambda x: '{:+.2e}'.format(x)}, linewidth=80)
    if verbose:
        print("raw_observables: fringepistons/rad", ff.nrm.fringepistons)
    utils.default_printoptions()
    return affine2d, psf_offset_find_rotation, ff.nrm.psf_offset, ff.nrm.fringepistons


def main(fitsimdir=None, oitdir=None, oifdir=None, ifn=None, oversample=3, mnem='', firstfew=None, verbose=False, usebp=True):
    """
    fitsimdir: string: dir containing data file
    ifn: str inout file name

    """

    np.set_printoptions(formatter={'float': lambda x: '{:+.2e}'.format(x)}, linewidth=80)
    if verbose:
        print("main: ", ifn)
    if verbose:
        print("main: fitsimdir", fitsimdir)
      
    aff, psf_offset_r, psf_offset_ff, fringepistons = raw_observables(fitsfn=ifn, 
                                                                      fitsimdir=fitsimdir, 
                                                                      oitdir=oitdir,
                                                                      oifdir=oifdir,
                                                                      oversample=oversample,
                                                                      firstfew=firstfew,
                                                                      usebp=usebp,
                                                                      verbose=verbose)
    print('aff', aff, 'psf_offset_r', psf_offset_r, 'psf_offset_ff', psf_offset_ff, 'fringepistons', fringepistons)

### Run ImPlaneIA

In [ ]:
mirdatafiles = ['jw01093012001_03102_00001_nis_calints.fits',
                'jw01093015001_03102_00001_nis_calints.fits']

# Choose FIRSTFEW = None to analyze all integrations
FIRSTFEW = 5
OVERSAMPLE = 7
print('FIRSTFEW', FIRSTFEW, 'OVERSAMPLE', OVERSAMPLE)


COUNT = 0
for fnmir in mirdatafiles:
    print('\nAnalyzing\n   ', COUNT, fnmir.replace('.fits', ''), end=' ')
    hdr = fits.getheader(datasuperdir / fnmir)
    print(hdr['FILTER'], end=' ')
    print(hdr['TARGNAME'], end=' ')
    print(hdr['TARGPROP'])
    # next line for convenient use in oifits writer which looks up target online
    catname = hdr['TARGPROP'].replace('-', '') # for target lookup on-line, otherwise UNKNOWN used
    fits.setval(datasuperdir / fnmir, 'TARGNAME', value=catname)
    fits.setval(datasuperdir / fnmir, 'TARGPROP', value=catname)
    
    usebp = False
        
    main(fitsimdir=datasuperdir,
         oitdir=datasuperdir / 'Saveoit',
         oifdir=datasuperdir / 'Saveoif',
         ifn=fnmir, 
         oversample=OVERSAMPLE, 
         mnem='',
         firstfew=FIRSTFEW,
         usebp=usebp,
         verbose=True) # verbose only has driver-function scope
    COUNT += 1

### Examine the output products

Analytical model is created and interferometric observables are calculated for each integration of the data. The output products are stored in a folder that has rootname of the file, jw01093012001_03102_00001_nis_calints for AB Dor and jw01093015001_03102_00001_nis_calints for HD37093. 

In [ ]:
# integration 0 (1st integration)
results_int0 = glob.glob(str(datasuperdir / 'Saveoit' / "jw01093012001_03102_00001_nis_calints" / "*00*"))

In [ ]:
results_int0

### Information about observables calculated from the 1st integration


```
- phases_00.txt: 35 fringe phases
- amplitudes_00.txt: 21 fringe amplitudes
- CPs_00.txt: 35 closure phases
- CAs_00.txt: 35 closure amplitudes
- fringepistons_00.txt: 7 pistons (optical path delays between mask holes)
- solutions_00.txt: 44 fringe coefficients of terms in the analytical model
- modelsolution_00.fits: analytical model
- n_modelsolution_00.fits: normalized analytical model
- residual_00.fits: data - model
- n_residual_00.fits: normalized residual

```

In [ ]:
cropped_data = fits.getdata(datasuperdir / 'Saveoit' / "jw01093012001_03102_00001_nis_calints" / "centered_0.fits")
model = fits.getdata(datasuperdir / 'Saveoit' / "jw01093012001_03102_00001_nis_calints" / "modelsolution_00.fits")
residual = fits.getdata(datasuperdir / 'Saveoit' / "jw01093012001_03102_00001_nis_calints" / "residual_00.fits")
n_residual = fits.getdata(datasuperdir / 'Saveoit' / "jw01093012001_03102_00001_nis_calints" / "n_residual_00.fits")

In [ ]:
f = plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.title("AB Dor cropped data", fontsize=12)
plt.imshow(cropped_data, origin='lower')
plt.subplot(1, 3, 2)
plt.title("AB Dor analytical model", fontsize=12)
plt.imshow(model, origin='lower')
plt.subplot(1, 3, 3)
plt.title("AB Dor residual (data - model)", fontsize=12)
plt.imshow(residual, origin='lower')

In [ ]:
plt.title("AB Dor normalized residual")
plt.imshow(n_residual, clim=(-0.03, 0.03), origin='lower')
plt.colorbar()

### OIFITS files for the target and calibrator

OIFITS is the standard data exchange format for Optical Interferometry.  It is based on the Flexible Image Transport System (FITS).  OIFITS files include data tables for storing interferometric observables, including squared visibilities and closure phases. 

In [ ]:
oifiles = sorted(glob.glob(str(datasuperdir / 'Saveoif' / "*oifits")))
oifiles

### Calibrate the closure phases and fringe amplitudes of target with the closure phases and fringe amplitudes of the calibrator.

This step is necessary to remove instrumental contribution to closure phases and fringe amplitudes.

In [ ]:
# Define the target and calibrator OIFITS files

targ_oifits = (datasuperdir / 'Saveoif' / 'jw01093012001_03102_00001_nis.oifits')
cal_oifits = (datasuperdir / 'Saveoif' / 'jw01093015001_03102_00001_nis.oifits')

# Produce a single calibrated OIFITS file

print("************  Running calibrate ***************")
print("Calibrating  AB Dor with HD37093")
calibrate_oifits(targ_oifits, cal_oifits, oifdir=str(datasuperdir / 'Saveoif'))


print("The output of calibrate is calibrated oifits file that will be used as an input to 3_niriss_ami_binary.ipynb.")

*Developer Note:*
The observable extraction performed in this notebook used only the first 5 integrations to save time while demonstrating the use of ImPlaneIA to reduce pipeline-calibrated observations. For accurate science use, we use all the integrations contained in the input data files. Therefore the input data for notebook 3 (3_niriss_ami_binary.ipynb) is slightly different from the output of 2_niriss_ami_binary.ipynb.

## Aditional Resources

- [JWST NIRISS AMI](https://jwst-docs.stsci.edu/near-infrared-imager-and-slitless-spectrograph/niriss-observing-modes/niriss-aperture-masking-interferometry)

## About this notebook

**Author:** Deepashri Thatte, Anand Sivaramakrishnan, Rachel Cooper, Jens Kammerer  
**Updated On:** 2022-09-16 

***

[Top of Page](#top)
<img style="float: right;" src="https://raw.githubusercontent.com/spacetelescope/notebooks/master/assets/stsci_pri_combo_mark_horizonal_white_bkgd.png" alt="Space Telescope Logo" width="200px"/> 